# PSU DS 200 Lab 8

In this lab, we will learn the following:
- Convert text (in tweets) into word frequency matrix representation.
- Construct a Decision Tree to classify tweets for sentiment prediction, using the word frequency representation.
- Evaluate the Decision Tree using Confusion Matrix, precision, recall, and f1 measure.
- Interpret the knowledge learned in the Decision Tree using rules.
- Compare the performance of two Decision Tree Models (generated using different depth) using f1 measures for both the training data and the testing data.

## Acknowledgement: This lab uses datascience Python module developed at UC Berkeley.
## The Source of Data used in this lab is an "Airline sentiment tweets" dataset from Kaggle.com.

In [1]:
!pip install datascience

     |████████████████████████████████| 51kB 2.1MB/s eta 0:00:011
     |████████████████████████████████| 92kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 13.0MB 58kB/s  eta 0:00:01    |███████▌                        | 3.0MB 7.8MB/s eta 0:00:02     |████████████████▎               | 6.6MB 7.8MB/s eta 0:00:01     |█████████████████████████▌      | 10.4MB 2.2MB/s eta 0:00:02     |██████████████████████████      | 10.6MB 2.2MB/s eta 0:00:02
     |████████████████████████████████| 215kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 29.2MB/s eta 0:00:01
     |████████████████████████████████| 92kB 7.0MB/s eta 0:00:011
  Created wheel for datascience: filename=datascience-0.15.3-cp35-none-any.whl size=44582 sha256=c4badf19dc48a4c6a31675bae27a4bf5f6f165df9951f8dffbc1ffd19c4f2eb2
  Stored in directory: /home/nbuser/.cache/pip/wheels/b8/37/0a/80274866028f6485c5957f0e1acf8e2b755fbe9dd0fd4ad275
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-an

In [4]:
import datascience
import numpy as np
import graphviz

from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

### Exercise 1 Load the Tweet File (3 points)
Use Table.read_table to load the airline_sentiment_small.csv file, whicch contain sentiment labels for airline-related tweets.

In [15]:
tweets= Table.read_table('airline_sentiment_small.csv', sep = ',')
tweets

tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
Tr_tweet_973,0,0.6671,Can't Tell,0.3651,United,Austin_Grisham,0,@united don't see a justifiable cost to get on an early ...,2/23/2015 11:14,PDX,Alaska
Tr_tweet_2085,1,1,nan,nan,United,eotoro,0,@united Kurt and the crew on UA1745 were amazing today. ...,2/21/2015 14:55,"iPhone: 0.000000,0.000000",Central Time (US & Canada)
Tr_tweet_2589,0,1,Customer Service Issue,0.6804,United,bugstory,0,@united the refund team is far from customer care courte ...,2/20/2015 11:15,"Baltimore, MD",Eastern Time (US & Canada)
Tr_tweet_445,1,1,nan,nan,Virgin America,Perceptions,0,@VirginAmerica really wish you'd fly out of #Fargo @farg ...,2/17/2015 10:33,"Fargo, ND ( & Tucson, AZ)",Central Time (US & Canada)
Tr_tweet_2302,0,1,Cancelled Flight,1,United,DNLee5,0,@united my flight out of BGM Cancelled Flightled last ni ...,2/21/2015 2:21,"New York, US; Tanzania, Africa",Central Time (US & Canada)
Tr_tweet_1111,1,0.6289,nan,0,United,sentientcheese,0,"@united been solved, they finally picked up the second ...",2/23/2015 7:20,nan,nan
Tr_tweet_1734,0,1,Cancelled Flight,0.684,United,ATL_Attorney,0,@united involuntarily bumped your passenger and can't gu ...,2/22/2015 7:54,"Atlanta, GA",Quito
Tr_tweet_138,0,1,Damaged Luggage,1,Virgin America,khartline,0,.@VirginAmerica does that mean you don't have a policy f ...,2/22/2015 12:27,Las Vegas,Pacific Time (US & Canada)
Tr_tweet_1963,0,0.684,Customer Service Issue,0.3499,United,Jo_Frost,0,@united when I read it say ' in some cases' can you plea ...,2/21/2015 19:12,Global,Quito
Tr_tweet_718,0,1,Customer Service Issue,1,United,fortytoo,0,"@united tried calling too, but w/25 min phn wait means w ...",2/23/2015 20:36,"WX report for 41 Mile, Hwy 50",Pacific Time (US & Canada)


In [16]:
X = tweets['text']
X

array(["@united don't see a justifiable cost to get on an early flight with seats. No airline charges to conveniently get their passengers in early",
       "@united Kurt and the crew on UA1745 were amazing today. They made my son's birthday with their kindness. Hope to fly with them again soon!",
       "@united the refund team is far from customer care courtesy they need training on work ethics. So it's like sir u r lying how can I help",
       "@VirginAmerica really wish you'd fly out of #Fargo @fargoairport those fares are amazings",
       '@united my flight out of BGM Cancelled Flightled last night, I get a hotel voucher, but had to pay for a cab back. Worse cab experience ever',
       '@united been solved,  they finally picked up the second time I called,  thanks for the response JH!  7:21 to dulles works!',
       "@united involuntarily bumped your passenger and can't guarantee them another flight for three days!",
       ".@VirginAmerica does that mean you don't have a polic

In [17]:
y = tweets['airline_sentiment']
y

array([0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0.

### Exercise 2 (5 points)
Choose a percentage (e.g., 0.2, 0.25, or 0.3) for splitting the data into training and testing.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
print(X_train)

['@united but then again, maybe the @BBBNE_SD_KS_IA would care more to hear what went on than whoever listens to issues you caused'
 "@united Great! I'm ready to go home."
 '@united I was protected on that flight by gate agent Kerry at LAS. She also did an excellent job getting me to my destination today. Thanks!'
 "@united Okay thanks if you could please update me. I was told at the airport someone would call me today but they haven't."
 "@united - you sure missed the mark on tonight's redeye from LAX to Chicago. What a mess! You can do better!"
 '@united worked out after all. Thanks for your immediate reply!!'
 '@united contd... I am being told that I am asking for a refund on a used ticket on a flight that got Cancelled Flightled how is the ticket used?'
 '@united how can you not put my bag on plane to Seattle. Flight 1212. Waiting  in line to talk to someone about my bag. Status should matter.'
 "@united can you help from there? UA4935 can't take off without missing galley cart tha

### Use CountVectorizer in sklearn to convert text of each tweet into a word frequency representation 
In Lab 3, we used CountVectorizer to convert a set of short texts into a word frequency matrix representation.  If you do not recall word frequency representation, you may want to revisit Lab 3 to refresh your memory.  In this lab, we are
also going to use CountVectorizer to convert a set of real tweets into a word frequency matrix representation.  We made two improvements to the way we convert text to word frequency matrix in Lab 3.
- Because real tweets contain non-alphanumeric characters such as emojis, we use a regular expression to extract all alphanumeric words, hashtags, and Twitter users (i.e., @...) mentioned in a tweet, ignore the other non-alphanumeric characters.  You do not need to understand the details of how this regulare expression is constructed.  You just need
to understand their function.
- Because the number of tweets are large, we do not want to include any word that occurs only in ONE tweet (why?). 
To do this, we use set "minimum document frequency" (min_df) to be 2.  "Documents" refers to, in this case, tweets.

In [20]:
count_vect = CountVectorizer(token_pattern= '((?:([@#]|[0-9]|[a-z]|[A-Z])+))', analyzer= 'word', min_df=2)

In [21]:
X_word_vect = count_vect.fit_transform(X_train)

In [22]:
print(X_word_vect.shape)

(480, 998)


### Exercise 3 (5 points)
- Choose the parameter for max_depth between 5 and 7 and min_samples_leaf to be 2.
- Evaluate it using confusion matrix, precision, recall, and f1 measure FOR TRAINING DATA.
- Evaluate it using confusion matrix, precision, recall, and f1 measure for TESTING DATA.
#### Put your answer in the text cell for Answer to Exercise 3 below, not here.

In [23]:
clf = tree.DecisionTreeClassifier(criterion='entropy', random_state = 100, max_depth=6, \
                                  min_samples_leaf =2)

In [24]:
clf.fit(X_word_vect, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [25]:
X_test_word_vect = count_vect.transform(X_test)

In [26]:
print(X_test_word_vect.shape)

(120, 998)


In [27]:
predicted_y = clf.predict(X_test_word_vect)
print(predicted_y)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [28]:
print(metrics.classification_report(y_test, predicted_y))

             precision    recall  f1-score   support

        0.0       0.89      0.99      0.94        85
        1.0       0.96      0.71      0.82        35

avg / total       0.91      0.91      0.90       120



In [29]:
metrics.confusion_matrix(y_test, predicted_y)

array([[84,  1],
       [10, 25]])

In [33]:
predicted_train_y = clf.predict(X_word_vect)

In [31]:
print(metrics.classification_report(y_train, predicted_train_y))

             precision    recall  f1-score   support

        0.0       0.84      0.99      0.91       315
        1.0       0.97      0.63      0.76       165

avg / total       0.88      0.87      0.86       480



In [32]:
metrics.confusion_matrix(y_train, predicted_train_y)

array([[312,   3],
       [ 61, 104]])

### Answer to Exercise 3 
- Performance Evaluation Using Training Data: The Precision is 0.88. The recall is 0.99. The f1-measure is 0.86.
- Performance Evaluation Using Testing Data: The Precision is 0.91. The recall is 0.91. The f1-measure is 0.90.

In [46]:
dot_data= tree.export_graphviz(clf, out_file=None, feature_names=count_vect.get_feature_names())

In [47]:
from graphviz import *

In [48]:
graph = graphviz.Source(dot_data)
graph

In [49]:
graph.render('8Graph')

'8Graph.pdf'

### Exercise 4 (5 points)
- Save the visualization of your decision tree in a file (i.e., fill in the file name in the code cell above).
- Choose a leaf node in the decision tree, then describe the classification rule from the root node to the leaf node.

### Answer to Exercise 4: 
- The value of the leaf node: 0.811
- The rule from the root node to the leaf node: True, True

### Exercise 5 (5 points)
Choose a max_detph of at least 10, construct the decision tree and evaluate its performance using both training data and testing data (similar to Exercise 3).  Provide your answer in the text cell below for "Answer to Exercise 5"

In [34]:
clf = tree.DecisionTreeClassifier(criterion='entropy', random_state = 100, max_depth=10, \
                                  min_samples_leaf =2)

In [35]:
clf.fit(X_word_vect, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [36]:
X_test_word_vect = count_vect.transform(X_test)

In [37]:
print(X_test_word_vect.shape)

(120, 998)


In [38]:
predicted_y = clf.predict(X_test_word_vect)
print(predicted_y)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [39]:
print(metrics.classification_report(y_test, predicted_y))

             precision    recall  f1-score   support

        0.0       0.91      0.99      0.95        85
        1.0       0.96      0.77      0.86        35

avg / total       0.93      0.93      0.92       120



In [40]:
metrics.confusion_matrix(y_test, predicted_y)

array([[84,  1],
       [ 8, 27]])

In [41]:
predicted_train_y = clf.predict(X_word_vect)

In [43]:
print(metrics.classification_report(y_train, predicted_train_y))

             precision    recall  f1-score   support

        0.0       0.85      1.00      0.92       315
        1.0       0.99      0.67      0.80       165

avg / total       0.90      0.89      0.88       480



In [50]:
metrics.confusion_matrix(y_train, predicted_train_y)

array([[314,   1],
       [ 54, 111]])

### Answer to Exercise 5 (5 points)
- Evaluation Using Training Data: The Precision is 0.90. The recall is 0.89. The f1-measure is 0.88. 
- Evaluation Using Testing Data: The Precision is 0.93. The recall is 0.93. The f1-measure is 0.92. 

In [ ]:
dot_data= tree.export_graphviz(clf, out_file=None, feature_names=count_vect.get_feature_names())

In [ ]:
graph = graphviz.Source(dot_data)
graph

In [51]:
graph.render('8graph2')

'8graph2.pdf'

### Exercise 6 (7 points)
Based on the results of Exercise 3 (we will referrted to it as Model A) and Exercise 5 (Model B), which one will you choose? Explain the rationale of your decision.
### Answer to Exercise 6:
- Choice of Model: Model B
- Rationale of Choice: The f1-measure tests are more accurate in Model B and have higher values.